<p style="font-weight:bold;"> <span style="font-size: 36px"> Interact With Import Scope Calculation</span> </p>

<br><br>
After successfully loading the parsed data in the dataSource, the import scopes for a specific partition can be re-evaluated in this notebook. 
The import scopes can then be displayed, examined and used for further calculations.

# Basic Initialization

In [0]:
#!import "../../Constants/CalculationEngine"

In [0]:
#!eval-notebook "../../Initialization/InitSystemorphBaseToMemory"
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.InitializeFrom(DataSource);

# Import of parsed data

In [0]:
Debug.Enable = true;

In [0]:
ProjectionCount = 5;

In [0]:
await Import.FromFile("../../Files/TransactionalData/Openings_FR_2020_12.csv")
            .WithFormat(ImportFormats.Opening)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync()

In [0]:

await Import.FromFile("../../Files/TransactionalData/NominalCashflows_FR_2020_12.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync()

# Args

## Primary Args

In [0]:
var reportingNode = "FR";
var year = 2020;
var month = 12;
var importFormat = ImportFormats.Cashflow;
var scenario = (string)null;

ImportArgs args = new ImportArgs(reportingNode, year, month, default(Periodicity), scenario, importFormat);
args

## Secondary Args

In [0]:
var allArgs = await GetAllArgsAsync(args, DataSource, ImportFormats.Cashflow);
allArgs

In [0]:
// var workspaceToCompute = Workspace.CreateNew();
// workspaceToCompute.InitializeFrom(DataSource);

In [0]:
// await ComputeAsync(args, Workspace, workspaceToCompute, true)

# Construct ImportStorage

In [0]:
var storage = new ImportStorage(args, DataSource, Workspace);
await storage.InitializeAsync();

# Create universe and Identities

In [0]:
var universe = Scopes.ForStorage(storage).ToScope<IModel>();
var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);

In [0]:
var idByDn = identities.ToDictionaryGrouped(x => x.DataNode, x => x.ToArray());
//idByDn

In [0]:
var riIds = idByDn["EY59R.2"];
//riIds

In [0]:
//storage.GetReinsuranceCoverage(riIds.First(), "EY59R.2")


In [0]:
var ivs = Scopes.ForIdentities(riIds, storage).ToScopes<ComputeAllScopes>().SelectMany(x => x.CalculatedIfrsVariables).ToArray();

In [0]:
int projection = 0;

In [0]:
Scopes.ForIdentities(riIds.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<LorecoCalculation>()
.Select(x => new {x.Identity, x.loReCoBoundaryValue, x.aggregatedLoReCoBoundary, x.reinsuranceCsm, x.AggregatedValue, x.Value})

In [0]:
var loreco = Scopes.ForIdentities(riIds.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<LorecoCalculation>()
.Where(x => x.Value != 0)
.Select(x => (x.Identity, x.Value))
.ToArray();

In [0]:
var csm = Scopes.ForIdentities(riIds.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<CsmReinsurance>()
.Where(x => x.Value != 0)
.Select(x => (x.Identity, x.Value))
.ToArray();

In [0]:
csm

In [0]:
loreco